## Examples of using libNeuroML to handle cell morphologies

First load a [NeuroML file containing a single cell](https://github.com/NeuralEnsemble/libNeuroML/blob/master/neuroml/examples/test_files/pyr_4_sym.cell.nml).

In [1]:
import neuroml
import neuroml.loaders as loaders

fn = '../neuroml/examples/test_files/pyr_4_sym.cell.nml'
doc = loaders.NeuroMLLoader.load(fn)
cell = doc.cells[0]
print("Loaded cell %s from: %s"%(cell.id,fn))

Loaded cell pyr_4_sym from: ../neuroml/examples/test_files/pyr_4_sym.cell.nml


### Summary

Print a brief summary of the cell properties

In [2]:
cell.summary()

*******************************************************
* Cell: pyr_4_sym
* Notes: A simplified pyramidal cell model with 9 compartments
* Segments: 9
* SegmentGroups: 18
*******************************************************


### Segments

Print information on the segments present

In [3]:
segments = cell.morphology.segments
print(segments)

for seg in segments:
    print('\nSegment %s, id: %s has proximal point: %s, distal: %s'%(seg.name, seg.id, seg.proximal, seg.distal))
    print(' Parent segment: %s%s'%(seg.parent.segments if seg.parent else 'None (root segment)', 
                                     '; fraction along: %s'%seg.parent.fraction_along if seg.parent else ''))

[<Segment|0|soma>, <Segment|1|apical0>, <Segment|2|apical2>, <Segment|3|apical3>, <Segment|4|apical4>, <Segment|5|apical1>, <Segment|6|basal0>, <Segment|7|basal1>, <Segment|8|basal2>]

Segment soma, id: 0 has proximal point: (0.0, 0.0, 0.0), diam 23.0um, distal: (0.0, 17.0, 0.0), diam 23.0um
 Parent segment: None (root segment)

Segment apical0, id: 1 has proximal point: (0.0, 17.0, 0.0), diam 6.0um, distal: (0.0, 77.0, 0.0), diam 6.0um
 Parent segment: 0; fraction along: 1.0

Segment apical2, id: 2 has proximal point: (0.0, 77.0, 0.0), diam 4.4um, distal: (0.0, 477.0, 0.0), diam 4.4um
 Parent segment: 1; fraction along: 1.0

Segment apical3, id: 3 has proximal point: (0.0, 477.0, 0.0), diam 2.9um, distal: (0.0, 877.0, 0.0), diam 2.9um
 Parent segment: 2; fraction along: 1.0

Segment apical4, id: 4 has proximal point: (0.0, 877.0, 0.0), diam 2.0um, distal: (0.0, 1127.0, 0.0), diam 2.0um
 Parent segment: 3; fraction along: 1.0

Segment apical1, id: 5 has proximal point: (0.0, 77.0, 0.0)

### Helper methods

Note that not all segments have an explicit **proximal**. This may be omitted if the **proximal** point is the same as the **distal** of the parent. For convenience, the helper method on **cell**, **get_actual_proximal()** can be used.

Note that these helper methods (and all the extra helper code that fed into the libNeuroML API) is specified [here](https://github.com/NeuralEnsemble/libNeuroML/blob/27b964c93c86796e5dcdc7d32c72e97e06ffccc2/neuroml/nml/helper_methods.py#L727), and is eventually generated into the main [nml.py](https://raw.githubusercontent.com/NeuralEnsemble/libNeuroML/master/neuroml/nml/nml.py).


In [4]:

for seg in segments:
    print('%s has proximal: %s, distal: %s'%(seg, 
                                             cell.get_actual_proximal(seg.id), 
                                            seg.distal))

<Segment|0|soma> has proximal: (0.0, 0.0, 0.0), diam 23.0um, distal: (0.0, 17.0, 0.0), diam 23.0um
<Segment|1|apical0> has proximal: (0.0, 17.0, 0.0), diam 6.0um, distal: (0.0, 77.0, 0.0), diam 6.0um
<Segment|2|apical2> has proximal: (0.0, 77.0, 0.0), diam 4.4um, distal: (0.0, 477.0, 0.0), diam 4.4um
<Segment|3|apical3> has proximal: (0.0, 477.0, 0.0), diam 2.9um, distal: (0.0, 877.0, 0.0), diam 2.9um
<Segment|4|apical4> has proximal: (0.0, 877.0, 0.0), diam 2.0um, distal: (0.0, 1127.0, 0.0), diam 2.0um
<Segment|5|apical1> has proximal: (0.0, 77.0, 0.0), diam 3.0um, distal: (-150.0, 77.0, 0.0), diam 3.0um
<Segment|6|basal0> has proximal: (0.0, 0.0, 0.0), diam 4.0um, distal: (0.0, -50.0, 0.0), diam 4.0um
<Segment|7|basal1> has proximal: (0.0, -50.0, 0.0), diam 5.0um, distal: (106.07, -156.07, 0.0), diam 5.0um
<Segment|8|basal2> has proximal: (0.0, -50.0, 0.0), diam 5.0um, distal: (-106.07, -156.07, 0.0), diam 5.0um


Other convenient helper methods include **get_segment_length()**, **get_segment_surface_area()** and **get_segment_volume()**, which also use the correct **proximal** point.

In [5]:

soma = cell.get_segment(0)
print('Segment %s has length: %s um, area: %s um^2, volume: %s um^3'%(soma, 
                                                  cell.get_segment_length(soma.id), 
                                                  cell.get_segment_surface_area(soma.id), 
                                                  cell.get_segment_volume(soma.id)))

Segment <Segment|0|soma> has length: 17.0 um, area: 1228.36272755 um^2, volume: 7063.08568343 um^3


### Segment Groups

Segment groups can refer to multiple segments (e.g. so that cannel densities can be placed across the cell).

They can include individual segments (e.g. `<member segment="8"/>`) and refer to other segment groups (e.g. `<include segmentGroup="soma"/>`).

In [6]:
for sg in cell.morphology.segment_groups:
    print(sg)
    if sg.id == 'dendrite_group':
        dendrite_group = sg

SegmentGroup: soma, 1 member(s), 0 included group(s)
SegmentGroup: apical0, 1 member(s), 0 included group(s)
SegmentGroup: apical2, 1 member(s), 0 included group(s)
SegmentGroup: apical3, 1 member(s), 0 included group(s)
SegmentGroup: apical4, 1 member(s), 0 included group(s)
SegmentGroup: apical1, 1 member(s), 0 included group(s)
SegmentGroup: basal0, 1 member(s), 0 included group(s)
SegmentGroup: basal1, 1 member(s), 0 included group(s)
SegmentGroup: basal2, 1 member(s), 0 included group(s)
SegmentGroup: all, 0 member(s), 9 included group(s)
SegmentGroup: soma_group, 0 member(s), 1 included group(s)
SegmentGroup: dendrite_group, 0 member(s), 8 included group(s)
SegmentGroup: apical_dends, 0 member(s), 5 included group(s)
SegmentGroup: middle_apical_dendrite, 0 member(s), 1 included group(s)
SegmentGroup: thalamic_input, 0 member(s), 1 included group(s)
SegmentGroup: basal_dends, 0 member(s), 3 included group(s)
SegmentGroup: basal_gaba_input, 0 member(s), 1 included group(s)
SegmentG

A helper method **get_all_segments_in_group()** will return a list of the segments in that group:

In [7]:
dend_segs = cell.get_all_segments_in_group(dendrite_group.id)
print(dend_segs)

[1, 2, 3, 4, 5, 6, 7, 8]
